# KM Master Method

In [ ]:
# track KM Master method
tracker_km_master = DataTracker("KM Master Method")
tracker_km_master.track(df_all_data, step_name="Initial Unique OP and Store", rows_unique=['OP', 'Toko'])

2025-12-10 06:55:46 - utils - INFO - Initialized DataTracker for: [KM Master Method]
2025-12-10 06:55:47 - utils - INFO - [KM Master Method] Step: Initial Unique OP and Store | Counts: **,*** | Change: +0 (+0.00%) | Retention: 100.00% | Step Time: 0.00s | Cumulative Time: 0.00s


## Main Method

In [ ]:
# get data with KM Master outside or equal to treshold for giving recommendation
min_km_master = 1
max_km_master = 1800
df_km_master = df_all_data[(df_all_data['KM Master'] < min_km_master) | (df_all_data['KM Master'] > max_km_master)]

# drop duplicate rows and sort based on OP and Toko
df_km_master = df_km_master.drop_duplicates(subset=['OP', 'Toko']).sort_values(by=['OP', 'Toko']).reset_index(drop=True)

# only keep relevant columns
df_km_master = df_km_master[['OP', 'Kode Zona', 'Toko', 'KM Master']]

# add recommendation column
df_km_master['KM Master Rekomendasi'] = np.nan

tracker_km_master.track(df_km_master, step_name="Filter with KM Master Outside Treshold", rows_unique=['OP', 'Toko'])

2025-12-10 06:55:47 - utils - INFO - [KM Master Method] Step: Filter with KM Master Outside Treshold | Counts: * | Change: -**,*** (-99.99%) | Retention: 0.01% | Step Time: 1.08s | Cumulative Time: 1.08s


In [ ]:
# get data from google sheets for recommendation
worksheet = sheets_loader(sheets_url=os.getenv('SHEETS_URL')).worksheet(config.UPDATED_WORKSHEET)
df_recommendation = pd.DataFrame(worksheet.get_all_records())
df_recommendation = df_recommendation.replace("", np.nan)
logger.info(f"Loaded {mask_numeric_value(f'{len(df_recommendation):,}')} rows of google sheets recommendation data")

# check if google sheet data is empty
if df_recommendation.empty:
    df_recommendation = pd.DataFrame(columns=['OP', 'Kode Zona', 'Toko', 'KM Master Rekomendasi'])

# check the data
check_content = df_km_master[['OP', 'Kode Zona', 'Toko']].isin(df_recommendation[['OP', 'Kode Zona', 'Toko']]).all().all()
check_recommendation = df_recommendation['KM Master Rekomendasi'].isna().sum()

if check_content: # if df_km_master content is in df_recommendation
    if check_recommendation > 0: # if there is OP and Store with no recommendation
        # update google sheet data with no KM Master recommendation
        df_km_master = df_km_master.where(df_km_master.notna(), "")
        sheets_updater(sheets_url=os.getenv('SHEETS_URL'), df=df_km_master)
        logger.warning(f"Update {config.UPDATED_WORKSHEET} with no KM Master recommendation")
        # hide values if configured
        if config.HIDE_VALUES:
            raise ValueError(f"Give recommendation to {config.UPDATED_WORKSHEET} in URL: {os.getenv('SHEETS_URL')[:30]} [Redacted]")
        else:
            raise ValueError(f"Give recommendation to {config.UPDATED_WORKSHEET} in URL: {os.getenv('SHEETS_URL')}")
    else:
        logger.info(f"Data in {config.UPDATED_WORKSHEET} is completed")
else:
    # update google sheet data with no KM Master recommendation
    df_km_master = df_km_master.where(df_km_master.notna(), "")
    sheets_updater(sheets_url=os.getenv('SHEETS_URL'), df=df_km_master)
    logger.warning(f"Update {config.UPDATED_WORKSHEET} with no KM Master recommendation")
    # hide values if configured
    if config.HIDE_VALUES:
        raise ValueError(f"Give recommendation to {config.UPDATED_WORKSHEET} in URL: {os.getenv('SHEETS_URL')[:30]} [Redacted]")
    else:
        raise ValueError(f"Give recommendation to {config.UPDATED_WORKSHEET} in URL: {os.getenv('SHEETS_URL')}")

tracker_km_master.track(df_km_master, step_name="Final Unique OP and Store", rows_unique=['OP', 'Toko'])            

2025-12-10 06:55:47 - google_sheets_io - INFO - Loading Google Sheets from URL: https://docs.google.com/spread [Redacted]...
2025-12-10 06:55:47 - google_sheets_io - INFO - Credentials loaded successfully
2025-12-10 06:55:47 - google_sheets_io - INFO - Google Sheets API authorized
2025-12-10 06:55:49 - google_sheets_io - INFO - Sheet opened successfully
2025-12-10 06:55:50 - __main__ - INFO - Loaded * rows of google sheets recommendation data
2025-12-10 06:55:50 - __main__ - INFO - Data in Rekomendasi Manual KM Master is completed
2025-12-10 06:55:50 - utils - INFO - [KM Master Method] Step: Final Unique OP and Store | Counts: * | Change: +0 (+0.00%) | Retention: 0.01% | Step Time: 3.59s | Cumulative Time: 4.67s


## KM Master Result

In [ ]:
# show summary
tracker_km_master.summary()

2025-12-10 07:16:45 - utils - INFO - [KM Master Method] DataTracker summary generated for 3 steps.


,Step,Counts,Change,Change (%),Retained (%),Duration (s),Cumulative Time (s)
0,Initial Unique OP and Store,"**,***",+0,+0.00,100.00,0.00,0.00
1,Filter with KM Master Outside Treshold,*,"-**,***",-99.99,0.01,1.08,1.08
2,Final Unique OP and Store,*,+0,+0.00,0.01,3.59,4.67


# KM Tempuh Method

## KM Max == 1 Category

## KM Max <> 1 Category

## Outlier Category

## KM Tempuh Result

# Master Zona Method

## Main Method

## Master Zona Result

# Results Summary